# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})

df

,data1,data2,key1,key2
0,0.063336,0.536544,a,one
1,-0.949618,0.025540,a,two
2,0.021390,-0.669408,b,one
3,0.694628,-1.102727,b,two
4,2.298068,0.828910,a,one


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a    0.470595
b    0.358009
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.180702
      two    -0.949618
b     one     0.021390
      two     0.694628
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,1.180702,-0.949618
b,0.021390,0.694628


In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.470595,0.463665
b,0.358009,-0.886068


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.180702  0.682727
     two  -0.949618  0.025540
b    one   0.021390 -0.669408
     two   0.694628 -1.102727

In [14]:
grouped.value_counts()

key1           
a      2.298068    1
      -0.949618    1
       0.063336    1
b      0.021390    1
       0.694628    1
dtype: int64

In [15]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0  0.063336  0.536544    a  one
4  2.298068  0.828910    a  one
('a', 'two')
      data1    data2 key1 key2
1 -0.949618  0.02554    a  two
('b', 'one')
     data1     data2 key1 key2
2  0.02139 -0.669408    b  one
('b', 'two')
      data1     data2 key1 key2
3  0.694628 -1.102727    b  two


In [17]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,0.021390,-0.669408,b,one
3,0.694628,-1.102727,b,two


In [24]:
df

,data1,data2,key1,key2
0,0.063336,0.536544,a,one
1,-0.949618,0.025540,a,two
2,0.021390,-0.669408,b,one
3,0.694628,-1.102727,b,two
4,2.298068,0.828910,a,one


In [25]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,0.470595,0.463665
b,0.358009,-0.886068


In [28]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,0.063336,0.536544,a,one,0.470595,0.463665
1,-0.949618,0.025540,a,two,0.470595,0.463665
4,2.298068,0.828910,a,one,0.470595,0.463665
2,0.021390,-0.669408,b,one,0.358009,-0.886068
3,0.694628,-1.102727,b,two,0.358009,-0.886068


In [31]:
#agg example function:
def peak_to_peak(arr):
    return arr.max() - arr.min()

# transform example function:
def center(arr):
    return arr - arr.mean()

# def apply example function:
def top(df, column, n=5):
    return df.sort_index(by=column)[-n:]

In [32]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4)
factor[:10]

0     (-0.146, 1.521]
1    (-1.813, -0.146]
2     (-0.146, 1.521]
3    (-1.813, -0.146]
4     (-0.146, 1.521]
5    (-1.813, -0.146]
6    (-1.813, -0.146]
7     (-0.146, 1.521]
8      (1.521, 3.188]
9     (-0.146, 1.521]
Name: data1, dtype: category
Categories (4, object): [(-3.487, -1.813] < (-1.813, -0.146] < (-0.146, 1.521] < (1.521, 3.188]]

In [ ]:
def get_stats(group):
    return {'min': group}

### 10-13-15

In [37]:
suits = ['H', 'S', 'C', 'D']
card_val = (range(1,11) + [10]*3)*4
base_names = ['A'] + range(2,11) + ['J', 'K', 'Q']
cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)

deck.head()

AH    1
2H    2
3H    3
4H    4
5H    5
dtype: int64

In [39]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

draw(deck)

6S     6
QH    10
6D     6
5C     5
AS     1
dtype: int64

In [40]:
get_suit = lambda card: card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  9C      9
   4C      4
D  7D      7
   6D      6
H  9H      9
   KH     10
S  10S    10
   9S      9
dtype: int64

In [46]:
deck.groupby(get_suit, group_keys=False).head()

AH    1
2H    2
3H    3
4H    4
5H    5
AS    1
2S    2
3S    3
4S    4
5S    5
AC    1
2C    2
3C    3
4C    4
5C    5
AD    1
2D    2
3D    3
4D    4
5D    5
dtype: int64

In [50]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data' : np.random.randn(8), 'weights' : np.random.rand(8)})

df.head()

,category,data,weights
0,a,-1.058035,0.691690
1,a,0.991291,0.846119
2,a,-1.231002,0.869476
3,a,1.446830,0.923561
4,b,-1.503767,0.420311


In [51]:
grouped = df.groupby('category')

get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.111932
b    0.006544
dtype: float64

In [70]:
df['blue'] = [0,0,0,0,.5,.5,.5, .5]

df.head()

,category,data,weights,blue
0,a,-1.058035,0.691690,0.0
1,a,0.991291,0.846119,0.0
2,a,-1.231002,0.869476,0.0
3,a,1.446830,0.923561,0.0
4,b,-1.503767,0.420311,0.5


In [72]:
df.pivot_table(index=['category', 'blue'])

,,data,weights
category,blue,,
a,0.0,0.037271,0.832711
b,0.5,-0.249895,0.518673


In [74]:
pd.crosstab(df.category, df.blue, margins=True)

blue,0.0,0.5,All
category,,,
a,4,0,4
b,0,4,4
All,4,4,8


In [75]:
dictex = {'a': 1, 'b': 2, 'c': 3}
dictex.get('d', 'd')

'd'